# Extending Data Model

**Prerequisite**:

- Access to a CDF Project.
- Know how to install and setup Python.
- Launch a Python notebook.

In this tutorial, we will show you how to extend a data model using the `CogniteCore` model as an example.


## Creating the Base Model

The `CogniteCore` model is an immutable model versioned by `Cognite`. To extend this model we first create a copy of it
in our own space, such that we can start to do modifications to it.

In [1]:
from cognite.neat import NeatSession, get_cognite_client

In [2]:
client = get_cognite_client(".env")

Found .env file in repository root. Loaded variables from .env file.


In [3]:
neat = NeatSession(client, verbose=True)

neat.read.cdf.data_model(("cdf_cdm", "CogniteCore", "v1"))

CDF <class 'tuple'> ('cdf_cdm', 'CogniteCore', 'v1') read successfully


[]

We have now downloaded the `CogniteCore` model, and need to run `Neat`'s verification on it

In [4]:
issues = neat.verify()

The `CogniteCore` model consists of 33 views, including views for 3D, Annotation, as well as several views that are used as interfaces.

We simpliy the core model by dropping these views.

In [6]:
neat.prepare.data_model.reduce(drop=["3D", "Annotation", "BaseViews"])

Next we will turn the `CogniteCore` model into the basis for our new model `ExtensionCore`

In [7]:
neat.prepare.data_model.to_extension(new_data_model_id=("sp_doctrino_space", "ExtensionCore", "v1"), prefix="MyOrg")

The new base model is now ready. We can now directly publish it to CDF 

In [9]:
neat.to.cdf.data_model()

,name,created
0,spaces,1.0
1,containers,NaN
2,views,11.0
3,data_models,1.0


<img src="../../artifacts/figs/core_extension.png" width="800">

Or we can dump it a spreadsheet and work on it from there.

In [11]:
neat.to.excel("my_extension.xlsx")

## Adding New Properties

In the spreadsheet, we have added the two properties `location` and `subLocation` to the `Asset` view. In addition, we have added our on container `myContainer` in the container sheet.

<img src="../../artifacts/figs/core_extension_excel.png" width="800">

We start a new `NeatSession` to load this new model publish it to CDF

In [21]:
neat = NeatSession(client)

In [13]:
neat.read.excel("my_extension.xlsx")

Excel <class 'str'> my_extension.xlsx read successfully


[]

In [14]:
neat.verify()

[]

In [18]:
neat.to.cdf.data_model()

,name,created
0,spaces,1
1,containers,1
2,views,11
3,data_models,1


This new model is now published to CDF with the two extra properties

<img src="../../artifacts/figs/core_extension_with_properties.png" width="400">

In [ ]:
# This cell is hidden so it does not show in the docs.
from cognite.neat._utils.cdf.loaders import SpaceLoader

SpaceLoader(client).clean("sp_doctrino_space");